In [ ]:
import numpy as np
import torch
import torch_geometric as pyg
import graphPINN
import math
import logging
from time import time
from tqdm.notebook import tqdm
from scipy.io import savemat, loadmat
import os

In [ ]:
folder = "C:\\Users\\NASA\\Documents\\ML_checkpoints\\2023-10-10\\"
if not os.path.exists(f'{folder}extraps\\'):
    os.makedirs(f'{folder}extraps\\')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
k = 100
l = 100
bd = 200
dataset = graphPINN.data.MHSDataset(f'D:\\v4_set_k={k}_l={l}_bd={bd}',k=k, l=l, bd=bd)

# trainset, validset, testset = torch.utils.data.random_split(dataset,[0.8, 0.1, 0.1],generator=torch.Generator().manual_seed(314))
trainset, validset, testset = torch.utils.data.random_split(dataset,[0.01, 0.005, 0.985],generator=torch.Generator().manual_seed(314))

In [ ]:
propdesign = [14,6,3]
convdesign = [18,12,3]
params = 14*6+6*3+18*12+12*3

propkernel = graphPINN.KernelNN(propdesign, torch.nn.ReLU)
propgraph = graphPINN.BDPropGraph(propkernel)
convkernel = graphPINN.KernelNN(convdesign, torch.nn.ReLU)
convgraph = graphPINN.ConvGraph(convkernel)
model = pyg.nn.DataParallel(graphPINN.FullModel(propgraph, convgraph))
model.load_state_dict(torch.load(f'{folder}_1691861661.7859707epoch-1.pt'))
model = model.module.to(device)

In [ ]:
params = 14*6+6*3+18*12+12*3
# params = 3132
model = torch.load(f'{folder}model_trainsize-58_k-{k}_params-{900}.pt')
model.to(device)
print(model)
# model = torch.load(f'{folder}_1680737885.339157model.pt').module

In [ ]:
set = 'train'

In [ ]:
for j in tqdm(range(0,5)):
#     print(trainset[j].to_homogeneous().to(device))
    if set == 'train':
        data = trainset[j].to(device)
    elif set == 'validate':
        data = validset[j].to(device)
    elif set == 'test':
        data = testset[j].to(device)
    else:
        data = dataset.get(j).to(device)

    print(j,data.sharpnum.cpu().numpy()[0])
    pred = model.forward(data)[0].detach()
    n0 = data['bd'].pos.shape[0]
    n  = data['in'].pos.shape[0] + n0

    Bml = np.concatenate((pred.cpu().numpy(), data['bd'].x[:,0:3].cpu().numpy()),0)
    Btn = np.concatenate((data['in'].y.cpu().numpy(), data['bd'].y.cpu().numpy()),0)
    nodes = np.concatenate((data['in'].pos.cpu().numpy(),
                            np.concatenate((data['bd'].pos.cpu().numpy(),np.zeros((n0,1))),1)
                           ),0)
    pres = np.concatenate((data['in'].x[:,3:6].detach().cpu().numpy(),data['bd'].x[:,3:6].detach().cpu().numpy()),0)
    sharp = data.sharpnum.cpu().numpy()[0]
    mat = {'Bml':Bml,
           'Btn':Btn,
           'nodes':nodes,
           'pres':pres,
           'sharp':sharp,
           'n0':n0,
           'n':n}
    savemat(f'{folder}\\extraps\\params-{params}_{set}{j}.mat',mat)

In [ ]:
print(pred.shape)
print(data['bd'].x)
print(data['bd'].y)
print(n0,n)

In [ ]:
n0 = data['bd'].pos.shape[0]
n  = data['in'].pos.shape[0] + n0

Bml = np.concatenate((pred.cpu().numpy(), data['bd'].x[:,0:3].cpu().numpy()),0)
Btn = np.concatenate((data['in'].y.cpu().numpy(), data['bd'].y.cpu().numpy()),0)
nodes = np.concatenate((data['in'].pos.cpu().numpy(),
                        np.concatenate((data['bd'].pos.cpu().numpy(),np.zeros((n0,1))),1)
                       ),0)
pres = np.concatenate((data['in'].x[:,3:6].detach().cpu().numpy(),data['bd'].x[:,3:6].detach().cpu().numpy()),0)
sharp = data.sharpnum.cpu().numpy()[0]

In [ ]:
for w in model.parameters():
    print(w.data)